In [1]:
import importlib
import numpy as np
import graph_analysis_functions as gaf
import pickle
importlib.reload(gaf)

<module 'graph_analysis_functions' from '/home/admin/Ana/MicroBrain/codes/Graph Analysis & by region/graph_analysis_functions.py'>

In [2]:
EDGE_NKIND_TO_LABEL = {
    2: "arteriole",
    3: "venule",
    4: "capillary"
}


In [ ]:
# --------------------------------------------------------------------------------------
# 1) IMPORT  MODULE 
# --------------------------------------------------------------------------------------
import graph_analysis_functions as gaf  


# --------------------------------------------------------------------------------------
# 2) USER INPUTS 
# --------------------------------------------------------------------------------------
DATA_PATH = "/home/admin/Ana/MicroBrain/output/graph_18_OutGeom_Hcut3_um.pkl"  
SPACE = "um"                        # "vox" or "um"
EPS_VOX = 2.0                        # ALWAYS in voxels (converted internally if SPACE="um")
DEGREE_THR = 4

# Box definition (center_vox from Paraview + size_um)
CENTER_VOX = [2100, 4200, 750]        # <-- from ParaView etc (in vox)
BOX_SIZE_UM = [400, 400, 400]        # <-- physical size

# Image resolution (for vox→µm conversions)
res_um_per_vox = np.array([1.625, 1.625, 2.5]) 

# Gaia density options
DO_GAIA = True
SLAB_UM = 50.0
SLAB_AXIS = "z"

# Redundancy (simple shortest paths)
DO_PATHS = True
K_PATHS = 5


# --------------------------------------------------------------------------------------
# 3) BUILD BOX CONSISTENT WITH SPACE
# --------------------------------------------------------------------------------------
if SPACE == "vox":
    box = gaf.make_box_in_vox(CENTER_VOX, BOX_SIZE_UM, res_um_per_vox=res_um_per_vox)
else:
    box = gaf.make_box_in_um(CENTER_VOX, BOX_SIZE_UM, res_um_per_vox=res_um_per_vox)

gaf.validate_box_faces(box)


# --------------------------------------------------------------------------------------
# 4) LOAD DATA + SYNC ATTRIBUTES
# --------------------------------------------------------------------------------------
data = gaf.load_data(DATA_PATH)            # dict with keys: graph, vertex, geom (+ _R optional)



data = gaf.sync_vertex_attributes_safe(data, space=SPACE)
G = data["graph"]

# keep only giant component
_, ncomp, data = gaf.single_connected_component(data)
G = data["graph"]

coords_attr = gaf.resolve_coords_attr(space=SPACE)
print("\n=== Graph ===")
print(G.vs.attributes())
print(G.es.attributes())
print("coords_attr:", coords_attr)
print("box:", box)



# --------------------------------------------------------------------------------------
# 5) CLASSIC QC: DUPLICATES + LOOPS
# --------------------------------------------------------------------------------------
dup_stats = gaf.duplicated_edge_stats(G)
loop_stats = gaf.loop_edge_stats(G)
print("\n=== QC ===")
print("Duplicates:", dup_stats)
print("Loops:", loop_stats)


# --------------------------------------------------------------------------------------
# 6) EDGE TYPES + LENGTH + DIAMETER
# --------------------------------------------------------------------------------------
print("\n=== Edge type composition ===")
edge_types = gaf.get_edges_types(G, label_dict=gaf.EDGE_NKIND_TO_LABEL, return_dict=True)

print("\n=== Mean length by nkind ===")
try:
    gaf.get_avg_length_nkind(G, space=SPACE)
except Exception as e:
    print("Length-by-nkind skipped:", e)

print("\n=== Diameter stats by nkind (+ plot) ===")
try:
    diam_stats = gaf.diameter_stats_nkind(G, label_dict=EDGE_NKIND_TO_LABEL, ranges=None, plot=True)
except Exception as e:
    print("Diameter stats skipped:", e)


# --------------------------------------------------------------------------------------
# 7) DEGREE + HDN SPATIAL PLOTS
# --------------------------------------------------------------------------------------
print("\n=== Degree / HDN ===")
uniq_deg, hdn_idx = gaf.get_degrees(G, threshold=DEGREE_THR)

# Spatial plots 
gaf.plot_degree_nodes_spatial(
    G,
    space=SPACE,
    coords_attr=coords_attr,
    degree_min=DEGREE_THR,
    degree_max=None,
    by_type=True,
    title=f"Nodes with degree ≥ {DEGREE_THR} ({SPACE})"
)

# --------------------------------------------------------------------------------------
# 8) BC (FACES ONLY) + TABLE + PLOTS
# --------------------------------------------------------------------------------------
print("\n=== BC faces (faces-only) ===")
bc_res = gaf.analyze_bc_faces(
    G,
    box=box,
    space=SPACE,
    coords_attr=coords_attr,
    eps=EPS_VOX,              # ALWAYS in vox; internally converted if SPACE="um"
    degree_thr=DEGREE_THR,
    return_node_ids=False
)

bc_df = gaf.bc_faces_table(bc_res, box_name="Box0")
print("\nBC table:\n", bc_df)

gaf.plot_bc_cube_net(
    bc_res,
    title=f"BC composition per face (space={SPACE})",
    show_unknown=False,
    show_high_degree=True
)

gaf.plot_bc_3_cubes_tinted(
    G,
    box,
    space=SPACE,
    coords_attr=coords_attr,
    eps=EPS_VOX
)


# --------------------------------------------------------------------------------------
# 9) GAIA-STYLE MICROSEGMENTS (density / volume fraction slabs) [requires atlas radii]
# --------------------------------------------------------------------------------------
if DO_GAIA:
    print("\n=== Gaia-style microsegments ===")
    ms = gaf.microsegments(data, space="um")
    gaf.count_microsegments_by_nkind(ms)

    # box must be µm for slabs
    if SPACE == "um":
        box_um = box
    else:
        box_um = gaf.make_box_in_um(center_vox=CENTER_VOX,box_size_um=BOX_SIZE_UM, res_um_per_vox=res_um_per_vox)

    try:
        slabs_df = gaf.vessel_vol_frac_slabs_in_box(ms, box_um, slab=SLAB_UM, axis=SLAB_AXIS)
        print("\nSlabs volume fraction:\n", slabs_df)
    except Exception as e:
        print("Slab density skipped (likely missing atlas radii):", e)


# --------------------------------------------------------------------------------------
# 10) REDUNDANCY 
# --------------------------------------------------------------------------------------
if DO_PATHS:
    #first k A→V shortest paths
    print("\n=== Simple A→V shortest paths (proxy) ===")
    paths_orig = gaf.av_paths_in_box(
        G,
        box=box,
        k=K_PATHS,
        space=SPACE,
        coords_attr=coords_attr,
        node_eps=0.0
    )
    print(f"Found {len(paths_orig)} paths (requested {K_PATHS}).")
    # plot paths in box
    if len(paths_orig):
        gaf.plot_av_paths_in_box(
            G,
            box=box,
            paths_orig=paths_orig,
            space=SPACE,
            coords_attr=coords_attr,
            node_eps=0.0
        )

print("\nAnalysis finished.")


KeyError: 'radii_atlas'

In [ ]:
# check differences radii and atlas 

import matplotlib.pyplot as plt

rv1 = data["vertex"]["radii"]
rv2 = data["vertex"]["radii_atlas"]

rg1 = data["geom"]["radii"]
rg2 = data["geom_R"]["radii_atlas_geom"]

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Vertex - image radii
axes[0,0].hist(rv1, bins=60, alpha=0.7)
axes[0,0].set_title("Vertex radii (image space)")
axes[0,0].set_xlabel("Radius")
axes[0,0].set_ylabel("Count")

# Vertex - atlas radii
axes[0,1].hist(rv2, bins=60, alpha=0.7)
axes[0,1].set_title("Vertex radii_atlas")
axes[0,1].set_xlabel("Radius")
axes[0,1].set_ylabel("Count")

# Geometry - image radii
axes[1,0].hist(rg1, bins=60, alpha=0.7)
axes[1,0].set_title("Geom radii (image space)")
axes[1,0].set_xlabel("Radius")
axes[1,0].set_ylabel("Count")

# Geometry - atlas radii (µm)
axes[1,1].hist(rg2, bins=60, alpha=0.7)
axes[1,1].set_title("Geom radii_atlas_geom_R (µm)")
axes[1,1].set_xlabel("Radius (µm)")
axes[1,1].set_ylabel("Count")

fig.suptitle("Comparison of radii distributions", fontsize=16)
plt.tight_layout(rect=[0,0,1,0.95])
plt.show()